Librerias necesarias

In [26]:
import pandas as pd
import os 
import random
import uuid
import string
from faker import Faker
from datetime import datetime, timedelta
from IPython.display import display, Markdown # Libreria unicamente para visualizar datos markdown en jupyter

Es necesario definir donde se almacenaran las bases de datos generadas para mantener un orden

In [27]:
# Carpeta para almacenar las bases de datos generadas
carpeta_database = 'database'

# Crear una carpeta si esta no existe
if not os.path.exists(carpeta_database):
  os.makedirs(carpeta_database)

# Path para guardar el archivo
path_info_clientes = os.path.join(carpeta_database, 'info_clientes.xlsx')
path_inventario = os.path.join(carpeta_database, 'inventario.xlsx')
path_demanda = os.path.join(carpeta_database, 'demanda.xlsx')
path_proveedores = os.path.join(carpeta_database, 'proveedores.xlsx')

# Generacion de base de datos para clientes



In [28]:
def generar_rut():
    # Genera un número aleatorio de 8 dígitos para el cuerpo del RUT 
    cuerpo_rut = str(random.randint(10000000, 20000000))

    # Calcula el dígito verificador utilizando el algoritmo de Módulo 11
    suma = sum(int(cuerpo_rut[i]) * (i % 6 + 2) for i in range(7))
    digito_verificador = 11 - (suma % 11)
    if digito_verificador == 11:
        digito_verificador = 0
    elif digito_verificador == 10:
        digito_verificador = 'K'

    # Combina el cuerpo y el dígito verificador para formar el RUT completo
    rut_completo = f"{cuerpo_rut}-{digito_verificador}"

    return rut_completo

In [29]:
fake = Faker('es_CL')

def generar_cliente():
  rut = generar_rut()
  nombre = fake.name()
  correo = fake.email()
  telefono = fake.phone_number()
  ciudad = fake.city()

  return {"Nombre": nombre, "Rut": rut, "Ciudad": ciudad, "Correo": correo, "Telefono": telefono}

cantidad_clientes = 10

cliente = [generar_cliente() for _ in range (cantidad_clientes)]

df = pd.DataFrame(cliente)

# Guardar el archivo
df.to_excel(path_info_clientes, index=False)

# Visualizar en markdown
display(Markdown(df.to_markdown()))

|    | Nombre                           | Rut        | Ciudad         | Correo                      | Telefono        |
|---:|:---------------------------------|:-----------|:---------------|:----------------------------|:----------------|
|  0 | Fernanda Camila González Briones | 13747053-K | Camiña         | victormunoz@example.net     | +56 58 272 1249 |
|  1 | Catalina Juana Rivera Núñez      | 11553274-3 | Algarrobo      | vcastro@example.org         | +56 51 320 9453 |
|  2 | Ignacio Tomás Díaz Ramírez       | 10939483-3 | Pudahuel       | gaston90@example.net        | +56 2 3609 5517 |
|  3 | Jacqueline Calderón González     | 18798542-8 | Río Bueno      | yvillar@example.com         | +56 73 218 5560 |
|  4 | Rocío Norma Cerda Figueroa       | 10326909-9 | Río Verde      | tobarhaydee@example.net     | +56 600 402 396 |
|  5 | Leonardo Luis González Bello     | 14768257-K | Nueva Imperial | mguzman@example.org         | +56 34 364 2579 |
|  6 | Ivonne Medina Hernández          | 12123552-3 | Putre          | antoniomoran@example.net    | +56 9 5475 2398 |
|  7 | Paulina Mireya Carrasco González | 17881920-8 | Coihaique      | barahonablanca@example.net  | +56 9 9408 3641 |
|  8 | Juan Sergio Méndez Henríquez     | 13053907-4 | Hualqui        | qgarcia@example.com         | +56 2 3902 9046 |
|  9 | Luis Lucas Mancilla Olguín       | 18812705-8 | Punta Arenas   | santiagovasquez@example.org | +56 61 237 0328 |

# Generacion base de datos para proveedores

In [30]:
fake_usa = Faker('en_US')
fake_brasil = Faker('pt_BR')
fake_italia = Faker('it_IT')

datos_falsos = [fake_usa, fake_brasil, fake_italia]

i = 0
def genenerar_proveedor():
  global i
  i = i + 1
  fake_extranjero = random.choice(datos_falsos)

  id_proveedor = f"P-{str(i).zfill(4)}"
  telefono_proveedor = fake_extranjero.phone_number()
  ciudad_proveedor = fake_extranjero.city()
  compania_proveedor = fake_extranjero.company()
  compra_proveedor = random.randint(10, 100)

  return {"ID Proveedor": id_proveedor, "Proveedor": compania_proveedor, "Ciudad": ciudad_proveedor, "Compra": compra_proveedor, "Telefono": telefono_proveedor}


num_proveedores = 50

producto = [genenerar_proveedor() for _ in range (num_proveedores)]

df = pd.DataFrame(producto)

#Guardar el archivo
df.to_excel(path_proveedores, index=False)

# Visualizar en markdown
display(Markdown(df.to_markdown()))


|    | ID Proveedor   | Proveedor                            | Ciudad                        |   Compra | Telefono              |
|---:|:---------------|:-------------------------------------|:------------------------------|---------:|:----------------------|
|  0 | P-0001         | Viola SPA                            | Malavicina                    |       85 | +39 361681811         |
|  1 | P-0002         | Clark-Duffy                          | Port Jennifer                 |       27 | 065.725.5217x7255     |
|  2 | P-0003         | Gonçalves Ltda.                      | Sales                         |       25 | +55 (041) 1372 8405   |
|  3 | P-0004         | Satta-Fornaciari s.r.l.              | Cunevo                        |       16 | 0721041974            |
|  4 | P-0005         | Giunti-Spinelli s.r.l.               | Masanti Di Sotto              |       11 | 094274097             |
|  5 | P-0006         | Mastandrea e figli                   | Trovo                         |       37 | 0771593212            |
|  6 | P-0007         | Costela Silva - ME                   | da Rosa                       |       17 | 71 8419 9250          |
|  7 | P-0008         | Liverotti-Novaro s.r.l.              | Casal Borsetti                |       64 | +39 035278848         |
|  8 | P-0009         | Paoletti-Foa e figli                 | Dogato                        |       74 | 07111785318           |
|  9 | P-0010         | Correia                              | Pinto de Cardoso              |       53 | 61 8945-5678          |
| 10 | P-0011         | da Mota                              | Gonçalves                     |       24 | +55 (061) 0062 8905   |
| 11 | P-0012         | Falcone, Baggio e Travia Group       | Pian Dei Ratti                |       86 | 05237552124           |
| 12 | P-0013         | Daniel-Martin                        | Hillfurt                      |       83 | 382.076.7091x850      |
| 13 | P-0014         | Vianello-Emanuelli SPA               | Bellocchi                     |       45 | 012377024             |
| 14 | P-0015         | Zanichelli-Nicoletti Group           | Manesseno                     |       14 | +39 01248175634       |
| 15 | P-0016         | Sales                                | Costela de Goiás              |       78 | 81 7639-8768          |
| 16 | P-0017         | Porto Barbosa - ME                   | Gonçalves Paulista            |       85 | +55 (084) 2792 1169   |
| 17 | P-0018         | Barros                               | Lopes da Serra                |       14 | (041) 7300 6936       |
| 18 | P-0019         | Johnson, Clarke and Duran            | South Shane                   |       30 | +1-581-189-8292x168   |
| 19 | P-0020         | Giunti, Zaccagnini e Ariasso s.r.l.  | Scalo Di Baragiano            |       12 | +39 36155214722       |
| 20 | P-0021         | Gibson Group                         | Kaylaborough                  |       74 | 991.647.7999x596      |
| 21 | P-0022         | Serao, Collina e Spinelli e figli    | Sotto Il Monte Giovanni Xxiii |       38 | +39 0403347345        |
| 22 | P-0023         | Wolf-King                            | Robinsonchester               |       60 | 323.063.7012x94911    |
| 23 | P-0024         | Freitas - ME                         | Araújo                        |       33 | (051) 3792 6557       |
| 24 | P-0025         | Favata s.r.l.                        | Gabbro                        |       68 | +39 3512158065        |
| 25 | P-0026         | Brooks, Carlson and Mckenzie         | Lake Jessicafurt              |       14 | +1-680-559-0888x54567 |
| 26 | P-0027         | Roberts, Garcia and Foley            | East Bradleyside              |       84 | (992)595-9358x72155   |
| 27 | P-0028         | Mendes - EI                          | Viana das Flores              |       25 | +55 21 7813-2838      |
| 28 | P-0029         | Panatta, Nicolini e Lippomano s.r.l. | Auronzo Di Cadore             |       23 | 32310852405           |
| 29 | P-0030         | da Cruz                              | Aragão do Campo               |       66 | (011) 0092-3978       |
| 30 | P-0031         | da Mota                              | Peixoto                       |       15 | +55 61 4043-3397      |
| 31 | P-0032         | Rocha S.A.                           | Barros de Minas               |       99 | 84 0608 1986          |
| 32 | P-0033         | Avila-Berry                          | Dawsonport                    |       87 | 322.044.1291          |
| 33 | P-0034         | Morales-Williams                     | South Maureenburgh            |       97 | 001-002-130-9536      |
| 34 | P-0035         | Smith-Maxwell                        | West Amber                    |       72 | +1-852-732-2915x5262  |
| 35 | P-0036         | Kim, Sutton and Smith                | North Ernesttown              |       91 | 353-783-6384x389      |
| 36 | P-0037         | Mennea e figli                       | Vicomoscano                   |       29 | 0774417266            |
| 37 | P-0038         | Peixoto                              | Alves                         |       13 | 0500-211-7289         |
| 38 | P-0039         | Guerra Ltd                           | Jeremiahville                 |       37 | 001-368-062-9511x4614 |
| 39 | P-0040         | Baroffio, Ligorio e Viola s.r.l.     | Gubbio                        |       82 | 349565123             |
| 40 | P-0041         | Gigli e figli                        | Pozzuoli                      |       64 | 04382773741           |
| 41 | P-0042         | Johnson Inc                          | North Daniel                  |       26 | 200.904.2455x609      |
| 42 | P-0043         | Campos Ferreira S.A.                 | da Cunha de Mendes            |       42 | +55 61 8753-6201      |
| 43 | P-0044         | Liverotti s.r.l.                     | Sant'Ambrogio Di Torino       |       59 | 3711226699            |
| 44 | P-0045         | Reed and Sons                        | Port Charlesbury              |       14 | +1-205-312-1404       |
| 45 | P-0046         | Taliani-Treccani s.r.l.              | Montelicciano                 |       52 | 0350964219            |
| 46 | P-0047         | Vercelloni-Seddio Group              | Torrebelvicino                |       89 | +39 3499521744        |
| 47 | P-0048         | Campise Group                        | Cimolais                      |       77 | +39 34294904986       |
| 48 | P-0049         | Lopez Group                          | Port Leonard                  |       18 | 158-680-3369x46003    |
| 49 | P-0050         | Lynch-Carter                         | Timothyport                   |       54 | 575-534-0478          |

# Generacion de base de datos para inventario

## Variables para los productos

In [31]:
# Listas de tipos de productos y sabores
tipo_prendas = ['Polera', 'Polerón', 'Camisa', 
                'Pantalón', 'Short', 'Vestido', 
                'Falda', 'Blusa', 'Camiseta', 
                'Jean', 'Calza', 'Legging', 
                'Abrigo', 'Chaleco', 
                'Traje de Baño', 'Pijama']

tallas = ['XS', 'S', 'M', 'L', 
          'XL', 'XXL', 'XXXL']

colores = ['rojo', 'azul', 'amarillo','verde', 
           'naranja', 'violeta','rosa', 'gris', 
           'marrón','blanco', 'negro', 'celeste']

tipo_materiales = ['algodon', 'poliester', 
                   'polar', 'mezclilla', 'viscosa']

marcas = ['Zara', 'H&M', 'Nike', 
          'Adidas', 'Gucci', 'Levi\'s', 
          'Forever 21', 'Puma', 
          'Calvin Klein', 'Tommy Hilfiger']

estampados = ['sin estampado', 'con estampado']




## Genera un ID unico para cada producto segun su tipo

In [32]:

conteo_por_prefijo = {}
def generar_codigo_producto(tipos_producto):
    # Obtén los tres primeros caracteres de la descripción
    prefijo = tipos_producto[:3].upper()

    if prefijo not in conteo_por_prefijo:
        conteo_por_prefijo[prefijo] = 1
    else:
        conteo_por_prefijo[prefijo] += 1

    codigo_str = str(conteo_por_prefijo[prefijo]).zfill(4)

    return f"{prefijo}{codigo_str}"


In [35]:
df_proveedores = pd.read_excel(path_proveedores)

In [40]:

def generar_descripcion():
  talla = random.choice(tallas)
  color = random.choice(colores)
  tipo_material = random.choice(tipo_materiales)
  marca = random.choice(marcas)

  return talla, color, tipo_material, marca


def generar_producto():

  proveedores = df_proveedores.sample(1).iloc[0]
  nombre_proveedor = proveedores['Proveedor']

  tipo_prenda = random.choice(tipo_prendas)
  talla, color, tipo_material, marca = generar_descripcion()
  descripcion = f"{tipo_prenda} color {color} talla {talla} marca {marca} de {tipo_material}"
  id_producto = generar_codigo_producto(tipo_prenda)
  precio_venta = fake.random_int(min=1000, max=10000)
  coste_compra = precio_venta - (precio_venta * random.uniform(0.4, 0.9))
  stock = random.randint(10, 100)

  return {"ID Producto": id_producto, 
          "Tipo de producto": tipo_prenda,
          "Descripcion": descripcion,
          "Proveedor": nombre_proveedor,
          "Stock": stock,
          "Stock de seguridad": stock * 0.2,
          "Precio de venta": precio_venta,
          "Coste de compra": coste_compra}

num_productos = 10

producto = [generar_producto() for _ in range (num_productos)]

df = pd.DataFrame(producto)

#Guardar el archivo
df.to_excel(path_inventario, index=False)

# Visualizar en markdown
display(Markdown(df.to_markdown()))

|    | ID Producto   | Tipo de producto   | Descripcion                                                  | Proveedor                  |   Stock |   Stock de seguridad |   Precio de venta |   Coste de compra |
|---:|:--------------|:-------------------|:-------------------------------------------------------------|:---------------------------|--------:|---------------------:|------------------:|------------------:|
|  0 | JEA0002       | Jean               | Jean color rosa talla XXL marca Zara de polar                | Guerra Ltd                 |      70 |                 14   |              2700 |          1307.74  |
|  1 | PIJ0005       | Pijama             | Pijama color rojo talla XS marca Levi's de poliester         | Giunti-Spinelli s.r.l.     |      10 |                  2   |              3475 |           482.544 |
|  2 | PAN0003       | Pantalón           | Pantalón color violeta talla M marca Tommy Hilfiger de polar | Zanichelli-Nicoletti Group |      62 |                 12.4 |              7483 |          2546.25  |
|  3 | CAM0006       | Camiseta           | Camiseta color amarillo talla S marca Zara de mezclilla      | Freitas - ME               |      59 |                 11.8 |              9582 |          4821.13  |
|  4 | CAL0002       | Calza              | Calza color rosa talla XS marca Zara de viscosa              | Clark-Duffy                |      96 |                 19.2 |              9382 |          2872.72  |
|  5 | POL0009       | Polera             | Polera color rosa talla XS marca Puma de mezclilla           | Gibson Group               |      92 |                 18.4 |              6757 |          1297.64  |
|  6 | CAM0007       | Camiseta           | Camiseta color gris talla M marca H&M de poliester           | Satta-Fornaciari s.r.l.    |      51 |                 10.2 |              8171 |           842.82  |
|  7 | JEA0003       | Jean               | Jean color gris talla M marca Tommy Hilfiger de mezclilla    | Sales                      |      18 |                  3.6 |              2972 |          1142.7   |
|  8 | FAL0004       | Falda              | Falda color marrón talla XXXL marca Adidas de viscosa        | Peixoto                    |      30 |                  6   |              4617 |          2074.83  |
|  9 | CHA0001       | Chaleco            | Chaleco color naranja talla L marca Nike de poliester        | Paoletti-Foa e figli       |      50 |                 10   |              4423 |          1476.75  |

# Ventas

## Leer base de datos de clientes
Esto es un paso necesario para porder asociar los clientes ya generados y de los cuales tenemos informacion, con los productos en inventario.

In [ ]:
df_clientes = pd.read_excel(path_info_clientes)
df_productos = pd.read_excel(path_inventario)

In [ ]:
# Establecer un rango realista para las fechas de compras
fecha_actual = datetime.now()

inicio_rango = fecha_actual - timedelta (days = 180)

Se genera una tabla que contenga una fecha realista para la demanda del producto, un nombre de cliente extraido de la base de datos y un tipo de producto que desee. Cabe destacar que de esta forma y usando la funcion 'random.choice' es que puede existir clientes que demanden mas de un tipo de producto, lo cual es completamente valido y cumple con una situacion realista. 

In [41]:
def generar_demanda():
  fecha_compra = fake.date_between_dates(date_start=inicio_rango, date_end=fecha_actual)
  

  producto = df_productos.sample(1).iloc[0]

  id_producto = producto['ID Producto']
  descripcion_producto = producto['Descripcion']
  precio_venta = producto['Precio de venta']

  cliente = df_clientes.sample(1).iloc[0]
  nombre_cliente = cliente['Nombre']
  rut = cliente['Rut']
  cantidad_productos = random.randint(1, 3)
  
  # Usar en caso de querer generar un ID de compra
  id_compra = uuid.uuid4().hex[:6]

  return {"ID Producto": id_producto, 
          "Descripcion": descripcion_producto,
          "Rut": rut,
          "Cantidad": cantidad_productos,
          "Precio de venta": precio_venta,
          "Precio total": precio_venta * cantidad_productos,
          "Fecha de compra": fecha_compra, 
          "Stock anterior": stock_anterior}
          

num_demanda_clientes = 10

demanda = [generar_demanda() for _ in range (num_demanda_clientes)]

df = pd.DataFrame(demanda)

# Guardar el archivo
df.to_excel(path_demanda, index=False)

# Visualizar en markdown
display(Markdown(df.to_markdown()))

|    | ID Producto   | Descripcion                                              | Rut        |   Cantidad |   Precio de venta |   Precio total | Fecha de compra   | Nombre                        |
|---:|:--------------|:---------------------------------------------------------|:-----------|-----------:|------------------:|---------------:|:------------------|:------------------------------|
|  0 | CAM0001       | Camiseta color azul talla L marca Adidas de polar        | 16626111-0 |          3 |              5977 |          17931 | 2023-11-08        | Ana Garrido                   |
|  1 | POL0001       | Polera color verde talla XXL marca Forever 21 de algodon | 16626111-0 |          2 |              1718 |           3436 | 2023-10-06        | Ana Garrido                   |
|  2 | BLU0001       | Blusa color azul talla XL marca Forever 21 de viscosa    | 11448889-4 |          1 |              2154 |           2154 | 2023-07-28        | Harold Pérez                  |
|  3 | CAM0001       | Camiseta color azul talla L marca Adidas de polar        | 17896332-2 |          1 |              5977 |           5977 | 2023-08-15        | Patricio Mauricio Ramos Díaz  |
|  4 | CAM0002       | Camiseta color rosa talla S marca H&M de algodon         | 18591985-0 |          3 |              1153 |           3459 | 2023-08-04        | Elena Madariaga Echeverría    |
|  5 | LEG0001       | Legging color amarillo talla L marca Forever 21 de polar | 10058937-K |          2 |              1689 |           3378 | 2023-07-11        | Juan Enrique Pardo Díaz       |
|  6 | CAM0001       | Camiseta color azul talla L marca Adidas de polar        | 17896332-2 |          3 |              5977 |          17931 | 2023-08-09        | Patricio Mauricio Ramos Díaz  |
|  7 | CAM0001       | Camiseta color azul talla L marca Adidas de polar        | 18591985-0 |          2 |              5977 |          11954 | 2023-09-17        | Elena Madariaga Echeverría    |
|  8 | LEG0001       | Legging color amarillo talla L marca Forever 21 de polar | 11380773-1 |          3 |              1689 |           5067 | 2023-09-06        | José Rodolfo Rodríguez Medina |
|  9 | VES0001       | Vestido color azul talla XXL marca Nike de poliester     | 10058937-K |          3 |              4000 |          12000 | 2023-10-14        | Juan Enrique Pardo Díaz       |

# Entradas